In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"

/Users/csuarezgurruchaga/Desktop/Digital-House/clase_12/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Data Wrangling

## Introducción

Data wrangling es el proceso de limpieza y unificación de conjuntos de datos desordenados y complejos para facilitar su acceso, exploración, análisis o modelización posterior.

Las tareas que involucra son
* Limpieza de datos
* Eliminación de registros duplicados
* Transformación de datos
* Discretización de variables
* Detección y filtro de outliers
* Construcción de variables dummies


## Dataset

En esta clase usaremos un dataset con info de películas que disponibiliza datos de movielens (https://movielens.org/).

https://grouplens.org/datasets/movielens/

http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Este conjunto de datos está conformado por varios archivos:
* **movies**: idPelicula, título y género; 

donde cada registro tiene los datos de una película

* **ratings**: idUsuario, idPelicula, rating, fecha; 

donde cada registro tienen la calificación otorgada por un usuario a una película

* **tags**: idUsuario, idPelicula, tag, fecha; 

donde cada registro tienen el tag que asignó un usuario a una película


## Imports

In [2]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos de movies, ratings y tags desde los archivos
* ../Data/movies.csv
* ../Data/ratings.csv
* ../Data/tags.csv

en las variables 
* data_movies
* data_ratings
* data_tags

Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna. 

Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [3]:
data_movies =pd.read_csv ('../Data/movies.csv', sep=',')
data_ratings=pd.read_csv('../Data/ratings.csv', sep=',')
data_tags=pd.read_csv('../Data/tags.csv', sep=',')

In [4]:
data_movies.info()
data_movies.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [5]:
data_ratings.info()
data_ratings.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [6]:
data_tags.info()
data_tags.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


## Ejercicio 2  - Registros duplicados

**2.a** Veamos si existen registros duplicados en el DataFrame data_tags considerando sólo las columnas "movieId", "tag", marcando como no duplicado la primera ocurrencia de un valor.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html


**2.b** Usemos el método `drop_duplicates` para obtener otro  `DataFrame` sin los casos duplicados considerando sólo las columnas "movieId", "tag". Usemos el método `duplicated` para verificar que el nuevo `DataFrame` efectivamente no tiene valores duplicados.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

In [7]:
data_tags.duplicated(["movieId", "tag"], keep='first').any()

True

In [8]:
data_tags_nodupl=data_tags.drop_duplicates(["movieId", "tag"], keep='first')

In [9]:
data_tags_nodupl.duplicated(["movieId", "tag"], keep='first').any()

False

## Ejercicio 3 - Transformar datos

Construyamos un diccionario que asocie un puntaje a una etiqueta.

Las etiquetas son:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4

Usemos el método `map` para crear una nueva columna en data (`rating_label`) que tenga las etiquetas asociadas al valor del campo `rating` para cada registro


In [10]:
bins = [0,3,4,5.1]

rating_label = pd.cut(data_ratings.rating, bins, labels = ['mala','regular','buena'], right=False)

data_ratings['rating_label'] = rating_label
data_ratings.sample(20)

,userId,movieId,rating,timestamp,rating_label
91163,590,38061,3.5,1260490848,regular
6492,45,104,5.0,950724870,buena
91251,591,1127,3.0,970524744,regular
96175,603,302,4.0,954482471,buena
89758,581,2762,4.5,1447007127,buena
24106,167,1220,4.5,1154718683,buena
27593,187,1209,5.0,1161849906,buena
39986,274,5463,2.5,1171828131,mala
58225,381,2081,3.5,1166971995,regular
73946,474,3011,4.0,1047054405,buena


Nota: esto ya sabíamos resolverlo usando máscaras booleanas

## Ejercicio 4 - Reemplazar valores

El método `replace` ofrece varias formas de efectuar reemplazos sobre una serie de Pandas:

* Un valor viejo por un valor nuevo.
    
* Una lista de valores viejos por un valor nuevo.
    
* Una lista de valores viejos por una lista de valores nuevos.
    
* Un diccionario que mapee valores nuevos y viejos.


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

**4.a - Una lista de valores viejos por un valor nuevo** 

Veamos cuáles son los tags que están asignados a una única película. 

Reemplacemos ese valor por "tag_que_no_funciona" y eliminemos registros duplicados considerando los campos "userId",  "movieId", "tag".

Ayuda: `value_counts`

In [11]:
tags=data_tags.tag.value_counts(ascending=True)

tags_df=tags.reset_index()

tags_valor_uno = tags_df.tag==1

tags_a_reemplazar=tags_df[tags_valor_uno]['index']
lista_a_reemplazar=list(tags_a_reemplazar)


df_4a=data_tags.replace(lista_a_reemplazar,"tag_que_no_funciona")

df_4a.duplicated(["userId", "movieId", "tag"]).any()

True

In [12]:
df_4a_fin=df_4a.drop_duplicates(["userId", "movieId", "tag"])

In [13]:
df_4a_fin.duplicated(["userId", "movieId", "tag"]).any()

False

**4.b - Una lista de valores viejos por una lista de valores nuevos**

Reemplacemos cada valor de tag, por la primera palabra que lo compone.

Para eso, creamos una serie con valores únicos con el valor del campo tag. 

Contruimos otra instancia de Series donde cada elemento sea la primera palabra del objeto Series anterior. Ayuda: listas por comprensión y `split`

Usamos replace para campiar el valor de cada tag por su primera palabra.


In [14]:
data_tags
tags_unicos = data_tags.tag.unique()
tags_unicos_2 =[ tags.split(' ')[0] for tags in tags_unicos]

tag_resumido = data_tags.replace(tags_unicos,tags_unicos_2)
# data_tags.head(3)
tag_resumido.head(3)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly,1445714996
2,2,60756,will,1445714992


**4.c - Un diccionario que mapee valores nuevos y viejos**

Reemplacemos los valores de tags 
* "Al Pacino" por "Pacino"
* "Leonardo DiCaprio" por "DiCaprio"
* "Tom Hanks" por "Hanks"
* "Martin Scorsese" por "Scorsese"

Contemos cuantas veces aparecen cada uno de los valores a reemplazar, y cuántas los valores de reemplazo. Ayuda: `value_counts`

Construyamos un diccionario con este mapeo y usemos el método `replace`

Volvamos a contar cuántas veces aparecen cada uno de los valores a reemplazar, y cuántas los valores de reemplazo.


## Ejercicio 5 - Discretizar variables

Vamos a volver a resolver el Ejercicio 3 usando el método `cut`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html

Defino los valores de corte:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4


## Ejercicio 6 - Detectar y filtrar outliers

No existe un criterio que sea válido en todos los casos para identificar los outliers. El criterio de mayor que el tercer cuartil más 1.5 veces el rango intercuartil o menor que el primer cuartil menos 1.5 veces el rango intercuartil (Q3 - Q1) surge de la distribución normal. En esa distribución el 99.7% de la población se encuentra en el rango definido por la media (poblacional) más menos 3 veces el desvío estándar (poblacional)

**Queremos ver cuáles son las películas que son outliers en cantidad de calificaciones.**

**6.a** Usando data_ratings eliminamos duplicados considerando las columnas "userId", "movieId". Esto lo hacemos para contar sólo una vez los votos de un usuario a una película.

**6.b** Sobre el DataFrame obtenido en el paso anterior, hacemos count agrupado por película. Esto nos da como resultado una instancia de Series que asignamos a la variable movie_votes_count.

**6.c** Calculemos los cuartilos de los valores de movie_votes_count y los valores que usaremos de umbral para determinar outliers.
(Ayuda: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html)

**6.d** Filtremos los datos de movie_votes_count excluyendo los outliers. 

**6.e** Comparemos movie_votes_count antes y después del filtro con:
* el método `describe`
* boxplots de seaborn

**6.f** Adicional: Miremos cuáles son los títulos de las cinco películas más votadas que son outliers de cantidad de calificaciones

In [15]:
data_ratings_sin_dup = data_ratings.drop_duplicates(["userId", "movieId"])

In [16]:
movie_votes_count=data_ratings_sin_dup.groupby('movieId').count()['userId']


In [18]:
q1 = movie_votes_count.quantile(0.25)
q2 = movie_votes_count.quantile(0.5)
q3 = movie_votes_count.quantile(0.75)


outlier_min_lvl = q1 - 1.5*(q3-q1)
outlier_max_lvl = q3 + 1.5*(q3-q1)

In [22]:
outliers=np.logical_or(movie_votes_count < outlier_max_lvl , outlier_min_lvl > movie_votes_count)

In [26]:
not_outlier = np.logical_not(outliers)
movie_votes_count_filtered=movie_votes_count[not_outlier]

In [27]:
movie_votes_count_filtered.describe()

count    1235.000000
mean       53.974089
std        40.529550
min        21.000000
25%        28.000000
50%        40.000000
75%        62.000000
max       329.000000
Name: userId, dtype: float64

## Ejercicio 7 - Variables categóricas y dummies

**7.a** Usando el método `get_dummies` con `drop_first = True` agreguemos al DataFrame data_ratings variables dummies que representen las categorias de rating_label

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html


**7.b** Comparemos las variables dummies generadas en el punto anterior con las que se generan usando `drop_first = False`. ¿Cuál es la diferencia? ¿Representan el mismo conjunto de valores posibles?

**7.c** Adicional: Cambienos las categorias que se muestran como resultado de `get_dummies` con `drop_first = True`. Ayuda: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categoricaldtype



En esta segunda solución (7.b), tenemos una columna para cada categoría de los valores originales. 

Los valores representados son exactamente los mismos que había en la columna original (como en las solución 7.a), pero una de las columnas es redundante porque se puede determinar su valor partiendo de los valores de las otras dos.

Las dos soluciones representan todas la categorias posibles de la variable original. 

Observemos que los valores (0,0,0), (0,1,1), (1,1,0), (1,0,1), (1,1) no representan una categoría en la variable original.